In [48]:
!pip install opensearch-py -U -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [61]:
!docker build -t task-store:latest . --quiet

sha256:6f12e61f90e071ee6193368cdc43e0d1a43e926a33b0d14b499bd0e916eed77d


In [54]:
!docker run -d -p 8080:8080 -e AWS_DEFAULT_REGION="us-west-2" -e AWS_REGION="us-west-2" --name test-task-store task-store:latest --host 0.0.0.0 --port 8080 --workers 3 --aoss-endpoint "os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com" --aoss-index "task-store-index"

8aa79c85151942973051bb74831d537a31443e23ad78459df3bbe5088a2c504d


In [40]:
!docker logs test-task-store

INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started parent process [1]
INFO:     Started server process [9]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [8]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [10]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [59]:
!docker kill test-task-store && docker rm -f test-task-store

test-task-store
test-task-store


In [38]:
!docker rm -f test-task-store

test-task-store


In [36]:
!docker context ls

NAME                             DESCRIPTION                               DOCKER ENDPOINT                                       ERROR
default                          Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                           
m5.large_i-0e2fa115a75372436 *                                             tcp://ip-10-3-1-173.us-west-2.compute.internal:1111   


In [24]:
host_ip = "ip-10-3-1-173.us-west-2.compute.internal"

In [41]:
%%bash -s $host_ip
curl ${1}:8080/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0   1888      0 --:--:-- --:--:-- --:--:--  2000


{"Health_Check":"200"}

In [55]:
import requests
import json

host_ip = "internal-codenator-899847730.us-west-2.elb.amazonaws.com"
port = 8010
task_desc = "Write Hello world code"
model_family = "bedrock"
model_name = "amazon.titan-embed-text-v1"
url = f"http://{host_ip}:{port}/invoke"
data = {
    "body": {"prompt": task_desc}, 
    "model_family": model_family, 
    "model_name": model_name
    
}
embedding = json.loads(requests.post(url=url, data=json.dumps(data)).text)["embedding"]

host_ip = "ip-10-3-1-173.us-west-2.compute.internal"
port = 8080

url = f"http://{host_ip}:{port}/save_task"
data = {
    "embedding": embedding,
    "script": "print(\"Hello World!\")",
    "task_desc": task_desc,
    "language": "python"
}
print(requests.post(url=url, data=json.dumps(data)).text)

{"status":"saved"}


In [56]:
url = f"http://{host_ip}:{port}/load_task"
data = {
    "embedding": embedding
}
print(requests.post(url=url, data=json.dumps(data)).text)

[{"embedding":[-0.35351562,0.30664062,0.20507812,0.026000977,0.953125,0.97265625,-0.13769531,-0.001159668,0.66015625,-0.5,-0.921875,-0.59375,0.65625,0.74609375,0.52734375,0.5,-0.62109375,0.83203125,-0.4609375,-0.5546875,-1.0625,0.890625,-0.9140625,-0.3046875,-0.14355469,0.018188477,-0.5078125,-0.51953125,0.21875,0.66796875,-0.96875,0.39453125,0.088378906,-1.2109375,-0.5078125,0.19335938,0.7421875,-1.4375,0.5234375,-0.1328125,0.6953125,0.7734375,-0.19335938,-1.6015625,-0.578125,0.22460938,0.5390625,-0.75,1.7109375,0.22460938,0.32421875,0.859375,-0.44140625,0.24316406,-0.55859375,0.421875,0.09814453,-0.42382812,0.6640625,-0.28515625,-0.7421875,0.7421875,-1.203125,0.3125,0.25195312,0.12597656,-0.6484375,-0.21777344,-0.48046875,0.46875,0.6328125,-0.005554199,0.032226562,1.1171875,-0.16601562,-0.45703125,0.24511719,0.091796875,0.73046875,1.8828125,1.0625,0.17285156,-0.25195312,-0.15527344,-0.6015625,0.33984375,-1.8515625,0.234375,0.0001077652,0.34570312,-0.43554688,-0.37890625,-0.32421875,0

In [49]:
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection
import boto3

# def push_to_aws():
host = "os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com"
credentials = boto3.Session().get_credentials()
region = "us-west-2" 

# Sign the credentials
auth = AWSV4SignerAuth(credentials, region, service="aoss")

# The OpenSearch client
client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

In [54]:
index_name = "task-store-index"
index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 512
    }
  },
  "mappings": {
    "properties": {
      "embedding": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "parameters": {},
          "space_type": "cosinesimil"
        }
      }
    }
  }
}
response = client.indices.create(index_name, body=index_body)
print(response)

In [60]:
client.indices.get_alias(index="*")

{'.opensearch_dashboards_1': {'aliases': {'.opensearch_dashboards': {}}},
 'task-store-index': {'aliases': {}}}

In [116]:
document = {
  "embedding": [0] * 1536,
  "task_desc": "Bennett Miller",
  "code": "2011"
}

response = client.index(
    index = "task-store-index",
    body = document
)
print(response)

{'_index': 'task-store-index', '_id': '1%3A0%3AekIuBowBqeZyZXfS5mmF', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}


In [58]:
client.delete(index="task-store-index", id="1%3A0%3AfEJdD4wBqeZyZXfS42mK")

{'_index': 'task-store-index',
 '_id': '1%3A0%3AfEJdD4wBqeZyZXfS42mK',
 '_version': 2,
 'result': 'deleted',
 '_shards': {'total': 0, 'successful': 0, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 0}

In [76]:
print(response)

{'_index': 'task-store-index', '_id': '1%3A0%3AeUL1BYwBqeZyZXfSyGnW', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}


In [127]:
query = {
  "size": 1,
  "query": {
      "knn": {
          "embedding": {
              "vector": [0] * 1536,
              "k": 1
          }
      }
  }
}

response = client.search(
    body = query,
    index = "task-store-index"
)
print(response)

{'took': 28, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.5, 'hits': [{'_index': 'task-store-index', '_id': '1%3A0%3AekIuBowBqeZyZXfS5mmF', '_score': 0.5, '_source': {'embedding': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [57]:
query = {
  "size": 100,
  "query": {
      "match_all": {
      }
  }
}

response = client.search(
    body = query,
    index = "task-store-index"
)
print(response)

{'took': 56, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'task-store-index', '_id': '1%3A0%3AfEJdD4wBqeZyZXfS42mK', '_score': 1.0, '_source': {'embedding': [-0.35351562, 0.30664062, 0.20507812, 0.026000977, 0.953125, 0.97265625, -0.13769531, -0.001159668, 0.66015625, -0.5, -0.921875, -0.59375, 0.65625, 0.74609375, 0.52734375, 0.5, -0.62109375, 0.83203125, -0.4609375, -0.5546875, -1.0625, 0.890625, -0.9140625, -0.3046875, -0.14355469, 0.018188477, -0.5078125, -0.51953125, 0.21875, 0.66796875, -0.96875, 0.39453125, 0.088378906, -1.2109375, -0.5078125, 0.19335938, 0.7421875, -1.4375, 0.5234375, -0.1328125, 0.6953125, 0.7734375, -0.19335938, -1.6015625, -0.578125, 0.22460938, 0.5390625, -0.75, 1.7109375, 0.22460938, 0.32421875, 0.859375, -0.44140625, 0.24316406, -0.55859375, 0.421875, 0.09814453, -0.42382812, 0.6640625, -0.28515625, -0.7421875, 0.7421875, -1.

In [90]:
response["hits"]

{'total': {'value': 1, 'relation': 'eq'},
 'max_score': 0.5,
 'hits': [{'_index': 'task-store-index',
   '_id': '1%3A0%3AeEL1BYwBqeZyZXfSjWl4',
   '_score': 0.5,
   '_source': {'embedding': [0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0

In [52]:
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection
from opensearch_dsl import Search
import boto3

# def push_to_aws():
host = f"os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com"
credentials = boto3.Session().get_credentials()
region = "us-west-2" 

# Sign the credentials
auth = AWSV4SignerAuth(credentials, region, service="aoss")

# The OpenSearch client
client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

my_index = "task-store-index"

try:
    response = client.indices.create(my_index)
    print("\nCreating index:")
    print(response)
except Exception as e:
    print(e)

#     files = glob.glob("/home/martin/json_directory/*.json")

#     for file in files:
#         with open(file, "r") as f:
#             print(f"Processing {file}")
#             data = payload_constructor(data=json.load(f), action={"index": {}})

#             response = client.bulk(body=data, index=my_index)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'task-store-index'}
